In [14]:


import pandas as pd 
import numpy as np 
import csv
import json 
import regex as re
from datetime import datetime
import matplotlib.pyplot as plt
from matplotlib.backends.backend_tkagg import FigureCanvasTkAgg
from matplotlib.figure import Figure
import tkinter as tk
from tkinter import ttk
from tkinter.filedialog import askopenfilename, asksaveasfilename


def csv_to_json(path):          #Function to load all CSV datasets and convert them to JSON file within the program folder
    with open(path) as file1:
        dict1 = csv.DictReader(file1)
        f_list = []
        for row in dict1:
            f_list.append(row)
        json_file = {"JSON" : f_list} #Make every CSV dataset a JSON file with JSON root node and list of dictionaries values as per JSON standards
        with open(f"{path[0:-4]}.json", "w") as file2: #keep the name of the CSV files but change the extension
            json.dump(json_file, file2, indent=2)

def cleaning(path):    #perform a general cleaning for CSV data sets, by removing inconsistances, spaces, zeros and null values to make them prepared for the analysis
    df = pd.read_csv(path)          #cleaning step should be done first before converting the CSV files to JSON
    df.replace(["none", 0 , " ", "0", "null", ""], np.nan, inplace = True)
    df = df.dropna(how="all")
    df.to_csv(path, index=False)

def data_manipulation1(path): #for request 1 and 2 in inspection table
    with open(path) as file1:
        data = json.load(file1)
        new_data = []
        for dict_element in data['JSON']:
            if dict_element["PROGRAM STATUS"] != "INACTIVE":
                new_data.append(dict_element)
        new_dict = { 'JSON' : new_data }

        for dict_element in new_data:   #managing error while extracting grayed infos in the "PE DESCRIPTION" column in case some fields have different format
            new_seating = re.split("\(.*\)" , dict_element["PE DESCRIPTION"])
            try:                        #managing error while extracting grayed infos in the "PE DESCRIPTION" column in case some fields have different format
                new_seating2 = re.search("\(.*\)" , dict_element["PE DESCRIPTION"]).group()
            except AttributeError :
                new_seating2 = "()"      #keep the grayed are only 2 brackets if it has different format

            try:                        #managing inconsistency in the "PE DESCRIPTION" column in case some fields have different format: like risk level
                dict_element["PE DESCRIPTION"] =  f"{new_seating[0]} {new_seating[1]}"
            except IndexError:
                dict_element["PE DESCRIPTION"] =  f"{new_seating[0]}"

            dict_element["New PE DESCRIPTION"] = new_seating2


        with open(f'{path[0:-5]}_new.json', 'w') as new_file:
            json.dump(new_dict, new_file, indent=2)

            

def data_manipulation2(path): #for request 2 in inventory JSON file only, as inventory table doesnt have "program status" like inspection 
    with open(path) as file1:
        data = json.load(file1)
       
        for dict_element in data['JSON']:
            new_seating = re.split("\(.*\)" , dict_element["PE DESCRIPTION"]) #using regular expression to extract seat number by splitting PE DESCRIPTION by ()
            try:                                    #try...except is used to control any cell has different format and to stop program from crashing
                new_seating2 = re.search("\(.*\)" , dict_element["PE DESCRIPTION"]).group() #putting seat number into new_seating2 by using RE
            except AttributeError :
                new_seating2 = "()"

            try:
                dict_element["PE DESCRIPTION"] =  f"{new_seating[0]} {new_seating[1]}" #modifying PE DESCRIPTION
            except IndexError:
                dict_element["PE DESCRIPTION"] =  f"{new_seating[0]}"

            dict_element["New PE DESCRIPTION"] = new_seating2 #seat number(grey area) is not into a new column called New PE DESCRIPTION


        with open(f'{path[0:-5]}_new.json', 'w') as new_file: #after the modfication is done, I saved the data in a new JSON file, this file I will be dealing
            json.dump(data, new_file, indent=2)                # with later as it has data manipulated and Inactive status have been filtered



def statistics(path): #Extracting mean, median and mode from inspection table for request 3
    with open(path) as file1:
        data = json.load(file1)
        data = data["JSON"]
    
        df = pd.DataFrame(data)
        df["year"]= df["ACTIVITY DATE"].str[-4:] #adding a new column called year to the df as year is always the last 4 chars in the activity date column
        df["year"] =  df["year"].astype(int)
        df["SCORE"]= df["SCORE"].replace("", 0)
        df["SCORE"] =  df["SCORE"].astype(float)
        grouped1 = df["SCORE"].groupby([df["year"], df["PE DESCRIPTION"]])#I grouped the score column by both years and type of vendor`s seating in PE DESCRIPTION

        df["Zip Codes"]= df["Zip Codes"].replace("", np.nan)
        df["Zip Codes"] = df["Zip Codes"].dropna()
        grouped2 = df["SCORE"].groupby([df["year"], df["Zip Codes"]])#I grouped the score column by both years and zip code

        return grouped1, grouped2 #returning both grouped objects to use them later for mean, median and mode


         
def violation_count(path): #a fuction to group violation column by VIOLATION CODE then return violation count
    with open(path) as file1:
        data = json.load(file1)
        data = data["JSON"]
        df = pd.DataFrame(data)
        grouped = df.groupby(df["VIOLATION CODE"])
        return grouped.count()


def plot_violation_count(count): # simply a function to plot violation counts to analyze itin reference to the establishments
    count.plot.bar()


def violations_correlation(path1, path2): # a function to visualize the correlation between violations code and zip code(which referes to areas)
    with open(path1) as file1:
        data1 = json.load(file1)
        data1 = data1["JSON"]
        df1 = pd.DataFrame(data1)
        df1 = df1[["SERIAL NUMBER", "FACILITY CITY", "Zip Codes"]]

        with open(path2) as file2:
            data2 = json.load(file2)
            data2 = data2["JSON"]
            df2 = pd.DataFrame(data2)
            df2 = df2[["SERIAL NUMBER", "VIOLATION CODE"]]

            df3 = pd.merge(df1,df2, on="SERIAL NUMBER") # I merged here both dataframes extracted from Inspections and violations in order to group them by facility
            grouped = df3.groupby(df3["FACILITY CITY"]) # I grouped by FACILITY CITY instead of Zip Codes as I can directly visualize the area that commited the                                                                highest violations

            return grouped.count()


#Below are the functions I called in order, I decided to leave them in case you want to try them in terminal rather than using GUI, you just can uncomment them by the same order:

#cleaning("Inspections.csv")
#cleaning('Inventroy.csv')
#cleaning("violations.csv")

#csv_to_json("Inspections.csv")
#csv_to_json("Inventroy.csv")
#csv_to_json("violations.csv")

#data_manipulation1("Inspections.json")
#data_manipulation2("Inventroy.json")

#statistics("Inspections_new.json")

#plot_violation_count(violation_count("violations.json"))

#violations_correlation("Inspections_new.json" , "violations.json")

#plot_violation_count(violations_correlation("Inspections_new.json" , "violations.json"))


#------------------------------------------------------------------------------------GUI-----------------------------------------------------------------------
inspections_csv_path = ""
inventory_csv_path = ""
violations_csv_path = ""


def func1(flag): #event handler fucntion to make other buttons active to respect the hererichy and the flow of the program, also it is the backbone of my GUI
                 #This function respond to all button clicks in the main window, except the last one(Data Analysis)
    global inspections_csv_path
    global inventory_csv_path
    global violations_csv_path


    if flag==0: #flag to successfully load Inspections file
        inspections_csv_path = askopenfilename(filetype=[("CSV Files", "*.csv")])
        if not inspections_csv_path: return
        if inspections_csv_path[-15:] != "Inspections.csv":
            lbl_welcome2["text"] = "Error, you didn load Inspections csv file!!"
            return
        else:
            lbl_welcome2["text"] = "Inspections is loaded successfully!!"


        btn_flags[0] = True
        
    elif flag==1: #flag to successfully loading Inventroy file
        inventory_csv_path = askopenfilename(filetype=[("CSV Files", "*.csv")])
        if not inventory_csv_path: return
        if inventory_csv_path[-13:] != "Inventroy.csv":
            lbl_welcome2["text"] = "Error, you didn load Inventroy csv file!!"
            return
        else:
            lbl_welcome2["text"] = "Inventroy is loaded successfully!!"


        btn_flags[1] = True

    elif flag==2: #flag to successfully loading violations file
        violations_csv_path = askopenfilename(filetype=[("CSV Files", "*.csv")])
        if not violations_csv_path: return
        if violations_csv_path[-14:] != "violations.csv":
            lbl_welcome2["text"] = "Error, you didn load violations csv file!!"
            return
        else:
            lbl_welcome2["text"] = "violations is loaded successfully!!"

        btn_flags[2] = True

    elif flag==3:    #flag to successfully clean the three loaded CSV files, the button cleaning is active only after the loading of 3 od them is done
        try:
            cleaning(inspections_csv_path)
            cleaning(inventory_csv_path)
            cleaning(violations_csv_path)
            lbl_welcome2["text"] = "Data is cleaned, you can now manipulate it!!"
            btn_manipulation['state'] = 'normal'
            return
    
        except Exception:
            lbl_welcome2["text"] = "Error loading files, please reload them!!"
            return

    elif flag==4: #flag to convert the 3 CSV files to JSON, then make the first and second requests for inspections and inventory tables as both of them only have
        try:      # Program status and PE Description
            csv_to_json(inspections_csv_path)
            csv_to_json(inventory_csv_path)
            csv_to_json(violations_csv_path)
            data_manipulation1(f"{inspections_csv_path[:-4]}.json")
            data_manipulation2(f"{inventory_csv_path[:-4]}.json")
        except Exception:
            lbl_welcome2["text"] = "Error Converting data to JSON and manipulate them, Make sure that you loaded the correct files!!"


        btn_analysis['state'] = 'normal'
        lbl_welcome2["text"] = "Data is manipulated, you can now analyis it!! \n From Inspection table all Inactive program status have been filtered and the numbers of seats have been extracted \n From Inventory table  the numbers of seated have been extracted as well "
        return

    if btn_flags==[True, True, True]: #these purpose of these flags is only to make the remaining buttons active to respect the program flow
        btn_clean['state'] = 'normal'
        lbl_welcome2["text"] = "Data now is loaded, you can now clean it!!"
        



def GUI_analysis_visualization(): #this function is invoked by the last button in the main window Data Analysis, it opens 1 subsequent new window
        new_window = tk.Toplevel(window)
        new_window.geometry("700x600")
        new_window.title("Data Analysis and Visualization")
        new_window.grid_columnconfigure([0], weight=1)
        
        lbl_1 = tk.Label(master = new_window, text= "Generating the mean, median, mode for the inspection score per year for : ")
        lbl_1.grid (row=5, column=0, sticky='w', padx=10)

        cbox = ttk.Combobox(master=new_window, values=("Type of Vedor's seating", "Zip Code"))
        cbox.current(0)
        cbox.grid(row=6, column=0, sticky='NSEW', padx=10)


        def btn_1_func(): #this function is invoked to calculate the mean, median and mode after selecting in GUI either "Type of Vedor's seating" or "Zip Code"
            nonlocal  cbox
            nonlocal txt_box_1
            statistic_reference = cbox.get()
            grouped1, grouped2 = statistics(f'{inspections_csv_path[0:-4]}_new.json')
            txt_box_1.delete('1.0', tk.END)

            if statistic_reference == "Type of Vedor's seating":
                txt_box_1.insert(tk.END, f"Mean:\n{grouped1.mean()}\n\n")
                txt_box_1.insert(tk.END,f"Mode:\n{grouped1.agg(lambda x:x.value_counts().index[0])}\n\n") # as there is no mode in Panda I aggregated in reference to 
                                                                                                          # value count
                txt_box_1.insert(tk.END,f"Median:\n{grouped1.median()}\n")
            elif statistic_reference == "Zip Code":
                txt_box_1.insert(tk.END, f"Mean:\n{grouped2.mean()}\n\n")
                txt_box_1.insert(tk.END,f"Mode:\n{grouped2.agg(lambda x:x.value_counts().index[0])}\n\n")
                txt_box_1.insert(tk.END,f"Median:\n{grouped2.median()}\n")
                
        btn_1 = tk.Button(master = new_window ,text='Calculate', command= btn_1_func )
        btn_1.grid (row=7, column=0, sticky='E', pady=5, padx=10)
        txt_box_1 = tk.Text(master =new_window); txt_box_1.grid (row=9, column=0, sticky='NSEW')


        def btn_2_func():#this function is invoked to show a plot of number of establishments that have committed each type of violations in a new window
            new_window2 = tk.Toplevel(window)
            new_window2.geometry("1920x600")
            new_window2.title("Plot")

            fig = Figure(figsize=(5,5), dpi=100)

            ax = fig.add_subplot(111)
            df = violation_count(f'{violations_csv_path[0:-4]}.json')
            df = df.iloc[:,0]
            df.plot.bar(ax=ax)

            canvas = FigureCanvasTkAgg(fig, master=new_window2)
            canvas.draw()
            canvas.get_tk_widget().pack(side="top",fill='both',expand=True)# I used pack instead of grid here in order to expand the plot to the whole screen



        def btn_3_func(): #this function is invoked to show a plot of correlation between number of violations per vendoe and zip code in a new window
            new_window3 = tk.Toplevel(window)
            new_window3.geometry("1920x600")
            new_window3.title("Plot")

            fig = Figure(figsize=(5,5), dpi=100)

            ax = fig.add_subplot(111)
            df = violations_correlation(f"{inspections_csv_path[0:-4]}_new.json" , f"{violations_csv_path[0:-4]}.json")
            df = df.iloc[:,1]
            df.plot.bar(ax=ax)
            
            print(df)
            canvas = FigureCanvasTkAgg(fig, master=new_window3)
            canvas.draw()
            canvas.get_tk_widget().pack(side="top",fill='both',expand=True)# I used pack instead of grid here in order to expand the plot to the whole screen

        lbl_2 = tk.Label(master = new_window, text= "Producing a graph counting the number of establishments that have commited each type of violation: ")
        lbl_2.grid (row=11, column=0, sticky='w', columnspan= 3, padx=10)
        btn_2 = tk.Button(master = new_window ,text='Plot' , command= btn_2_func)
        btn_2.grid (row=12, column=0, sticky='NSEW', pady=5, padx=10)

        lbl_3 = tk.Label(master = new_window, text= "Producing a graph counting the number of establishments that have commited each type of violation in every"                                                       +"city(by zip code): ")
        lbl_3.grid (row=13, column=0, sticky='w', columnspan= 3, padx=10)
        btn_3 = tk.Button(master = new_window ,text='Plot' , command= btn_3_func )
        btn_3.grid (row=14, column=0, sticky='NSEW', pady=5, padx=10)








window = tk.Tk()

lbl_welcome = tk.Label(text= "Welcome to Client Brief Program"); lbl_welcome.config(font=("Courier", 30, "bold" ))
lbl_welcome.config(fg="#0000FF"); lbl_welcome.config(bg="yellow")
lbl_welcome.grid (row=4, column=0, sticky='NSEW', columnspan= 3)
lbl_welcome2 = tk.Label(text= "Please note: you need to load all Data and clean them before proceeding to visualization then analysis")
lbl_welcome2.config(fg="red")
lbl_welcome2.grid(row=5, column=0, sticky='NSEW', columnspan= 3)
lbl_welcome3 = tk.Label(text= "Please choose the below available options to unlock others"); lbl_welcome3.config(font=( "arial ",18, "bold" ))
lbl_welcome3.grid(row=6, column=0, sticky='NSEW', columnspan= 3)

btn_flags = [False, False, False]
btn_load_inspections = tk.Button(text='Load Inspections', command= lambda : func1(0) )
btn_load_inspections.grid (row=7, column=0, sticky='NSEW')
btn_load_inventory = tk.Button(text='Load Inventory', command= lambda : func1(1) )
btn_load_inventory.grid (row=7, column=1, sticky='NSEW')
btn_load_violations = tk.Button(text='Load Violations', command= lambda : func1(2))
btn_load_violations.grid(row=7, column=2, sticky='NSEW')

btn_clean = tk.Button(text='Clean Loaded Data', state='disabled', command= lambda : func1(3) )
btn_clean.grid (row=8, column=0, sticky='NSEW', columnspan= 3)
btn_manipulation = tk.Button(text='Prepare Data for Visualization', state='disabled', command= lambda : func1(4))
btn_manipulation.grid (row=9, column=0, sticky='NSEW', columnspan= 3)

btn_save =  tk.Button(text='Save Current State')

btn_analysis = tk.Button(text='Data Analysis', state='disabled', command= GUI_analysis_visualization)
btn_analysis.grid (row=10, column=0, sticky='NSEW', columnspan= 3)

btn_statistics = tk.Button(text='Apply')

btn_visualiations = tk.Button(text='Apply')


window.geometry("800x600")
window.title("Big Data Analysis program")

window.grid_columnconfigure([0,1,2],weight=1)
window.grid_rowconfigure([4,5,6,7,8,9,10],weight=1)

window.mainloop()